## Import necessary module

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta

## Create function to scraping gold price in VietNam according to date

In [6]:
def get_price_gold(day, month, year):
    day = 13
    month = 6
    year = 2022
    URL = f"https://giavang.pnj.com.vn/history?gold_history_day={day}&gold_history_month={month}&gold_history_year={year}"
    r = requests.get(URL)
    soup = BeautifulSoup(r.content, 'lxml')
    table = soup.find('table')
    rows = table.find_all('tr')
    portlet_body_div = soup.find('div', id="p_p_id_com_pnj_gold_price_web_SearchGoldPriceResultPortlet_INSTANCE_3WGHuiSEaY89_")
    # print(portlet_body_div)
    # row = portlet_body_div.fi('tr')
    sjc_table = None
    purchase_prices = []
    sell_prices = []
    update_times = []
    # Find all tables in the parsed HTML
    tables = portlet_body_div.find_all('table')
    # print(tables)
    
    # Loop through each table to find the one related to HCM and SJC
    for table in tables:
        # Check if the table header contains 'Lịch sử giá vàng TPHCM' (assuming it's HCM)
        header = table.find('th', class_='style1')
        # print(header.text)
        if header and ('Lịch sử giá vàng TPHCM' in header.text):
            # Found the correct table for HCM (TPHCM), now find SJC rows
            sjc_rows = table.find_all('tr')  # Only direct children rows
            # print(sjc_rows)
            for row in sjc_rows:
                cells = row.find_all('td', recursive=False)
                if len(cells) == 4 and cells[0].text.strip() == 'SJC':
                    # Extract data
                    purchase_price = cells[1].text.strip()
                    sell_price = cells[2].text.strip()
                    update_time = cells[3].text.strip()
                    
                    # Append data to lists
                    purchase_prices.append(purchase_price)
                    sell_prices.append(sell_price)
                    update_times.append(update_time)
                    df = pd.DataFrame({
                        'Purchase Price': purchase_prices,
                        'Sell Price': sell_prices,
                        'Update Time': update_times
                    })
                    break

## Get gold price now

In [7]:
date = end_date = datetime.now()
URL = f"https://giavang.pnj.com.vn/history?gold_history_day={date.day}&gold_history_month={date.month}&gold_history_year={date.year}"
r = requests.get(URL)
soup = BeautifulSoup(r.content, 'lxml')
table = soup.find('table')
rows = table.find_all('tr')

# Define lists to store the extracted data
data = []

# Loop through the rows and extract data for each gold price entry
for row in rows[1:]:  # Skip the header row
    cells = row.find_all('td')
    if len(cells) == 5:  # Only process rows with 5 cells
        khu_vuc = cells[0].get_text(strip=True)
        loai_vang = cells[1].get_text(strip=True)
        gia_mua = cells[2].get_text(strip=True)
        gia_ban = cells[3].get_text(strip=True)
        thoi_gian_cap_nhat = cells[4].get_text(strip=True)
        
        data.append([khu_vuc, loai_vang, gia_mua, gia_ban, thoi_gian_cap_nhat])
    elif len(cells) == 4:  # Process rows with rowspan
        loai_vang = cells[0].get_text(strip=True)
        gia_mua = cells[1].get_text(strip=True)
        gia_ban = cells[2].get_text(strip=True)
        thoi_gian_cap_nhat = cells[3].get_text(strip=True)
        
        data.append([khu_vuc, loai_vang, gia_mua, gia_ban, thoi_gian_cap_nhat])

# Create a DataFrame from the extracted data
df = pd.DataFrame(data, columns=['Region', 'GoldType', 'PurchasePrice', 'SellingPrice', 'UpdateTime'])

# Save the DataFrame to a CSV file
df.to_csv('gold_prices.csv', index=False)
df

,Region,GoldType,PurchasePrice,SellingPrice,UpdateTime
0,TPHCM,PNJ,73.300,75.000,17/06/2024 08:13:04
1,TPHCM,SJC,74.980,76.980,17/06/2024 08:13:04
2,Hà Nội,PNJ,73.300,75.000,17/06/2024 08:13:04
3,Hà Nội,SJC,74.980,76.980,17/06/2024 08:13:04
4,Đà Nẵng,PNJ,73.300,75.000,17/06/2024 08:13:04
5,Đà Nẵng,SJC,74.980,76.980,17/06/2024 08:13:04
6,Miền Tây,PNJ,73.300,75.000,17/06/2024 08:13:04
7,Miền Tây,SJC,74.980,76.980,17/06/2024 08:13:04
8,Tây Nguyên,PNJ,73.300,75.000,17/06/2024 08:13:04
9,Tây Nguyên,SJC,74.980,76.980,17/06/2024 08:13:04


## Function to get SJC gold price acording to date
`Note:` Some date didn't have date

In [8]:
def get_gold_price(date,month,year):
    URL = f"https://giavang.pnj.com.vn/history?gold_history_day={day}&gold_history_month={month}&gold_history_year={year}"
    r = requests.get(URL)
    soup = BeautifulSoup(r.content, 'lxml')
    portlet_body_div = soup.find('div', id="p_p_id_com_pnj_gold_price_web_SearchGoldPriceResultPortlet_INSTANCE_3WGHuiSEaY89_")
    sjc_table = None
    purchase_prices = []
    sell_prices = []
    update_times = []
    # Find all tables in the parsed HTML
    tables = portlet_body_div.find_all('table')
    # print(tables)
    # Loop through each table to find the one related to HCM and SJC
    for table in tables:
        # Check if the table header contains 'Lịch sử giá vàng TPHCM' (assuming it's HCM)
        header = table.find('th', class_='style1')
        # print(header.text)
        if header and ('Lịch sử giá vàng TPHCM' in header.text):
            # Found the correct table for HCM (TPHCM), now find SJC rows
            sjc_rows = table.find_all('tr')  # Only direct children rows
            # print(sjc_rows)
            for row in sjc_rows:
                cells = row.find_all('td', recursive=False)
                if len(cells) == 4 and cells[0].text.strip() == 'SJC':
                    # Extract data
                    purchase_price = cells[1].text.strip()
                    sell_price = cells[2].text.strip()
                    update_time = cells[3].text.strip()
                    
                    # Append data to lists
                    purchase_prices.append(purchase_price)
                    sell_prices.append(sell_price)
                    update_times.append(update_time)
                    df = pd.DataFrame({
                        'Purchase Price': purchase_prices,
                        'Sell Price': sell_prices,
                        'Time': update_times
                    })
                    return df
                    break

## Get gold price in 10 earlist year to analyze

In [ ]:
start_date = datetime(year=2022, month=1, day=1)
end_date = datetime(year=2022, month=12, day=31)
gold_price = pd.DataFrame(columns=['Purchase Price','Sell Price','Time'])
print(gold_price)
current_date = start_date
while current_date <= end_date:
    day = current_date.day
    month = current_date.month
    year = current_date.year
    df = get_gold_price(day,month,year)
    dt = [gold_price,df]
    gold_price = pd.concat(dt).reset_index(drop=True)
    current_date += timedelta(days=1)

Empty DataFrame
Columns: [Purchase Price, Sell Price, Time]
Index: []


In [10]:
gold_price.head()

,Purchase Price,Sell Price,Time
0,66.400,67.200,03/01/2023 08:21:58
1,66.300,67.200,04/01/2023 08:19:28
2,66.300,67.200,05/01/2023 08:12:49
3,66.000,66.900,06/01/2023 08:20:11
4,66.300,67.200,07/01/2023 08:14:08


In [11]:
gold_price['Date'] = pd.to_datetime(gold_price['Time'],format='%d/%m/%Y %H:%M:%S').dt.date
gold_price = gold_price.drop(columns='Time')

In [12]:
gold_price.to_csv('sjc_gold_2023.csv')